<a href="https://colab.research.google.com/github/kanewilliams/ZoomCamp2024-HW/blob/main/Module02_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework for Module 2
## Stock Analytics Zoomcamp 2024
[Link](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2024/homework%202.md)

In [1]:
!pip install yfinance
# IMPORTS
import numpy as np
import pandas as pd

#Financial Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

**Q1: IPO Filings Web Scraping and Data Processing - What's the total sum ($m) of 2023 filings that happened on Fridays?**
---



In [2]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)
ipo_dfs = pd.read_html(response.text)

ipo_filings = ipo_dfs[0]
ipo_filings.info()
ipo_filings

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
320,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
321,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
322,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
323,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [3]:
import re

def extract_avg_price(input_string):
    # Extracts all prices in the string e.g. $0.52 or $26 (NOT $.61).
    # Case:
    # (i)    No Number: -              => NaN
    # (ii)       Price: $4.61          => 4.61
    # (iii) Two Prices: $8.00 - $10.00 => $9.00
    price_strings = re.findall(r'\$(?:\d+)(?:\.\d+)?', input_string)
    avg_price = np.mean([float(price.strip("$")) for price in price_strings])
    return avg_price

In [4]:
ipo_filings['Filing Date'] = pd.to_datetime(ipo_filings['Filing Date'], format="%b %d, %Y")
ipo_filings['Shares Offered'] = pd.to_numeric(ipo_filings['Shares Offered'], errors='coerce')
ipo_filings['Avg_price'] = ipo_filings["Price Range"].apply(lambda x:extract_avg_price(x))
ipo_filings['Shares_offered_value'] = pd.to_numeric(ipo_filings['Shares Offered'] * ipo_filings['Avg_price'], errors='coerce')

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [5]:
ipo_filings.info()
ipo_filings.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           325 non-null    datetime64[ns]
 1   Symbol                325 non-null    object        
 2   Company Name          325 non-null    object        
 3   Price Range           325 non-null    object        
 4   Shares Offered        252 non-null    float64       
 5   Avg_price             258 non-null    float64       
 6   Shares_offered_value  249 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 17.9+ KB


Filing Date              0
Symbol                   0
Company Name             0
Price Range              0
Shares Offered          73
Avg_price               67
Shares_offered_value    76
dtype: int64

In [6]:
'''Find the total sum in $m (millions of USD, closest INTEGER number)
   for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4).
   You should see 32 records in total, 25 of it is not null. '''

friday_2023_filings = ipo_filings[(ipo_filings['Filing Date'].dt.year == 2023) & (ipo_filings['Filing Date'].dt.dayofweek == 4)]
total_sum_millions = int(friday_2023_filings['Shares_offered_value'].sum() / 1000000)
print(f"Total sum of 2023 filings that happened on Fridays: ${total_sum_millions}m")

Total sum of 2023 filings that happened on Fridays: $285m


**Q2. IPOs "Fixed days hold" strategy - Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**
---



Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

- You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
- You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.

Additional:

- You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
- What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [7]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)
ipo_dfs = pd.read_html(response.text)
ipos_2024 = ipo_dfs[0]

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)
ipo_dfs = pd.read_html(response.text)
ipos_2023 = ipo_dfs[0]

stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)
stacked_ipos_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.65,20.98%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.15,-21.25%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$33.77,5.25%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$49.83,77.96%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$19.17,0.53%
...,...,...,...,...,...,...
213,"Jan 25, 2023",QSG,QuantaSing Group Ltd,$12.50,$3.19,-74.88%
214,"Jan 20, 2023",CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.48,-90.28%
215,"Jan 13, 2023",SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.61,150.73%
216,"Jan 13, 2023",ISRL,Israel Acquisitions Corp,$10.00,$10.92,9.20%


In [8]:
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format="%b %d, %Y")
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'])
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100

In [9]:
stacked_ipos_df = stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01']
stacked_ipos_df = stacked_ipos_df[stacked_ipos_df['Symbol'] != 'RYZB']

In [10]:
stacked_ipos_df['IPO Date']

33    2024-02-27
34    2024-02-22
35    2024-02-21
36    2024-02-16
37    2024-02-14
         ...    
213   2023-01-25
214   2023-01-20
215   2023-01-13
216   2023-01-13
217   2023-01-13
Name: IPO Date, Length: 184, dtype: datetime64[ns]

In [11]:
stacked_ipos_df = stacked_ipos_df.set_index('IPO Date')
stacked_ipos_df.index

DatetimeIndex(['2024-02-27', '2024-02-22', '2024-02-21', '2024-02-16',
               '2024-02-14', '2024-02-09', '2024-02-09', '2024-02-09',
               '2024-02-09', '2024-02-08',
               ...
               '2023-02-03', '2023-02-01', '2023-01-27', '2023-01-27',
               '2023-01-26', '2023-01-25', '2023-01-20', '2023-01-13',
               '2023-01-13', '2023-01-13'],
              dtype='datetime64[ns]', name='IPO Date', length=184, freq=None)

In [12]:
stacked_ipos_df

,Symbol,Company Name,IPO Price,Current,Return
IPO Date,,,,,
2024-02-27,SMXT,"SolarMax Technology, Inc.",4.00,10.34,1.6713
2024-02-22,VHAI,Vocodia Holdings Corp,4.25,0.13,-0.9689
2024-02-21,DYCQ,DT Cloud Acquisition Corporation,10.00,10.16,0.0140
2024-02-16,CHRO,Chromocell Therapeutics Corp,6.00,1.84,-0.7233
2024-02-14,UMAC,"Unusual Machines, Inc.",4.00,1.09,-0.7350
...,...,...,...,...,...
2023-01-25,QSG,QuantaSing Group Ltd,12.50,3.19,-0.7488
2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",5.00,0.48,-0.9028
2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",15.00,37.61,1.5073


In [13]:
end_date = date(2024, 3, 1)
ticker_symbols = list(stacked_ipos_df['Symbol'])
OHLCV_df = yf.download(ticker_symbols, end= end_date, progress= True)
OHLCV_df

[*********************100%%**********************]  184 of 184 completed
ERROR:yfinance:
4 Failed downloads:
ERROR:yfinance:['DYCQ', 'LEGT', 'JVSA']: Exception("%ticker%: Data doesn't exist for startDate = -1407024000, endDate = 1709269200")
ERROR:yfinance:['PTHR']: Exception('%ticker%: No timezone found, symbol may be delisted')


Price      Adj Close                                                     \
Ticker          AACT       AESI    AFJK        AHR    AITR  AIXI   ALCY   
Date                                                                      
2009-12-29       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2009-12-30       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2009-12-31       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2010-01-04       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2010-01-05       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
...              ...        ...     ...        ...     ...   ...    ...   
2024-02-23     10.55  18.320000  10.175  13.365355  10.240  1.87  10.55   
2024-02-26     10.54  18.330000  10.180  13.031221  10.240  1.90  10.56   
2024-02-27     10.55  19.809999  10.174  13.316218  10.242  2.43  10.56   
2024-02-28     10.48  19.299999  10.180  13.404666  10.240  2.06  10.54   
2024-02-29     10.53  18.860001  10.154  13.434148  10.240  1.98  10.55   

Price                            ...      Volume                      \
Ticker        ANL   ANRO   ANSC  ...        VHAI      VSME      VTMX   
Date                             ...                                   
2009-12-29    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2009-12-30    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2009-12-31    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2010-01-04    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2010-01-05    NaN    NaN    NaN  ...         NaN       NaN       NaN   
...           ...    ...    ...  ...         ...       ...       ...   
2024-02-23  7.151  13.72  10.14  ...         NaN  166900.0  125000.0   
2024-02-26  7.150  15.31  10.15  ...   2444600.0  326800.0   77300.0   
2024-02-27  7.150  14.36  10.15  ...  10196900.0  963200.0  114200.0   
2024-02-28  7.150  15.33  10.13  ...   1554300.0  393900.0  165400.0   
2024-02-29  7.150  15.40  10.14  ...   1853300.0  177900.0   90100.0   

Price                                                                        
Ticker          WBUY       WLGS      WRNT YGFGF     YIBO      ZJYL      ZKH  
Date                                                                         
2009-12-29       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2009-12-30       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2009-12-31       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2010-01-04       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2010-01-05       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
...              ...        ...       ...   ...      ...       ...      ...  
2024-02-23  225200.0   107100.0  128200.0   0.0  35700.0  323200.0  54100.0  
2024-02-26  137400.0  1060100.0   38900.0   0.0  20400.0  578500.0  42700.0  
2024-02-27  161600.0   375400.0  147600.0   0.0  19300.0  218900.0  40000.0  
2024-02-28  182900.0   266700.0  187200.0   0.0   9700.0  200900.0  55200.0  
2024-02-29   88600.0   139100.0   62700.0   0.0  11400.0  120300.0  76600.0  

[3566 rows x 1104 columns]

In [14]:
OHLCV_df.describe()

Price    Adj Close                                                           \
Ticker        AACT        AESI       AFJK        AHR       AITR        AIXI   
count   180.000000  246.000000  27.000000  16.000000  41.000000  246.000000   
mean     10.314455   18.012292  10.174370  13.221015  10.210463    3.861199   
std       0.125658    2.068070   0.008563   0.239321   0.024179    1.960839   
min      10.100000   15.115661  10.154000  12.834673  10.140000    1.460000   
25%      10.200000   16.646191  10.170000  13.021394  10.200000    2.102500   
50%      10.290000   17.189692  10.175000  13.247426  10.210000    2.620000   
75%      10.426250   19.265822  10.180000  13.404666  10.230000    5.598750   
max      10.550000   23.631262  10.190000  13.630697  10.242000   10.040000   

Price                                                ...        Volume  \
Ticker        ALCY         ANL       ANRO      ANSC  ...          VHAI   
count   164.000000  105.000000  19.000000  40.00000  ...  4.000000e+00   
mean     10.371994    9.324810  16.077895  10.12325  ...  4.012275e+06   
std       0.107671    1.313951   2.625976   0.02043  ...  4.139646e+06   
min      10.180000    7.111000  13.000000  10.07000  ...  1.554300e+06   
25%      10.310000    8.600000  14.090000  10.10750  ...  1.778550e+06   
50%      10.355000    9.130000  15.330000  10.13000  ...  2.148950e+06   
75%      10.430000   10.100000  17.764999  10.14000  ...  4.382675e+06   
max      10.600000   15.000000  21.799999  10.15000  ...  1.019690e+07   

Price                                                                         \
Ticker          VSME          VTMX          WBUY          WLGS          WRNT   
count   1.070000e+02  1.680000e+02  9.100000e+01  2.170000e+02  1.520000e+02   
mean    2.120798e+06  1.589054e+05  1.067866e+06  7.502235e+05  3.875770e+05   
std     7.001629e+06  2.994474e+05  2.325684e+06  3.689922e+06  1.196544e+06   
min     0.000000e+00  1.780000e+04  1.170000e+04  1.300000e+03  1.200000e+04   
25%     2.142000e+05  5.830000e+04  1.901000e+05  3.130000e+04  5.445000e+04   
50%     3.714000e+05  9.340000e+04  3.461000e+05  7.290000e+04  9.825000e+04   
75%     9.756000e+05  1.600000e+05  5.665500e+05  2.417000e+05  2.393500e+05   
max     5.861290e+07  3.372400e+06  1.315850e+07  3.809550e+07  1.239670e+07   

Price                                                             
Ticker         YGFGF           YIBO          ZJYL            ZKH  
count   2.330000e+02      25.000000  2.330000e+02      51.000000  
mean    6.258541e+04   86412.000000  2.415530e+06   67570.588235  
std     1.719811e+05  180463.090224  7.631175e+06   70216.363603  
min     0.000000e+00    5800.000000  4.000000e+03    5000.000000  
25%     1.040000e+04   18200.000000  3.860000e+05   40100.000000  
50%     3.910000e+04   35700.000000  1.064000e+06   53400.000000  
75%     7.050000e+04   68900.000000  2.288000e+06   71750.000000  
max     2.503800e+06  875500.000000  1.134760e+08  506300.000000  

[8 rows x 1104 columns]

In [15]:
def growth_rate_table(day):
    '''Returns a table of future growth rate after `day` days for each stock for each date.
       e.g. return_table['STOCK'].iloc(DATE) returns the `day`th future
       growth rate for STOCK at DATE.'''
    return OHLCV_df['Adj Close'].shift(-day) / OHLCV_df['Adj Close']

In [16]:
def find_growth_rate(stock, date, growth_rate_table):
    return growth_rate_table[stock][date]

In [17]:
for i in range(1, 31):
    growth_table = growth_rate_table(i)
    stacked_ipos_df[f'growth_future_{i}'] = stacked_ipos_df['Symbol'].apply(
                                                lambda x: find_growth_rate(x, stacked_ipos_df.index[stacked_ipos_df['Symbol'] == x][0], growth_table)
                                                )
stacked_ipos_df.sort_values(by=['Symbol'])

,Symbol,Company Name,IPO Price,Current,Return,growth_future_1,growth_future_2,growth_future_3,growth_future_4,growth_future_5,...,growth_future_21,growth_future_22,growth_future_23,growth_future_24,growth_future_25,growth_future_26,growth_future_27,growth_future_28,growth_future_29,growth_future_30
IPO Date,,,,,,,,,,,,,,,,,,,,,
2023-04-21,AACT,Ares Acquisition Corporation II,10.0,10.62,0.0620,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-09,AESI,Atlas Energy Solutions Inc.,18.0,22.50,0.2500,0.973451,0.943363,0.979941,0.946903,0.951032,...,1.023009,1.049557,1.050737,1.058997,1.076106,1.074926,1.057817,1.066077,1.056637,1.072566
2023-12-04,AFJK,"Aimei Health Technology Co., Ltd Ordinary Share",10.0,10.28,0.0280,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-07,AHR,"American Healthcare REIT, Inc.",12.0,13.99,0.1658,0.987897,0.992436,0.994705,1.009834,1.040847,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-09,AITR,AI TRANSPORTATION ACQUISITION CORP,10.0,10.32,0.0320,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-25,WRNT,Warrantee Inc.,4.0,0.34,-0.9158,0.788372,0.572093,0.508139,0.441860,0.372093,...,0.241860,0.223256,0.226744,0.209302,0.244186,0.217674,0.209302,0.209302,0.213953,0.225814
2023-03-28,YGFGF,YanGuFang International Group Co. Ltd.,4.0,0.00,-0.9996,0.975064,0.976864,0.997686,1.002571,0.940874,...,0.950386,0.904884,1.002571,0.866324,0.868895,0.750643,0.758355,0.754499,0.830334,0.943445
2024-01-25,YIBO,Planet Image International Limited,4.0,2.19,-0.4538,0.974910,0.931900,0.953405,0.867384,0.821505,...,0.931900,0.982079,1.014337,0.982079,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
stacked_ipos_df.drop(["IPO Price", "Current", "Return"], axis=1).describe()

,growth_future_1,growth_future_2,growth_future_3,growth_future_4,growth_future_5,growth_future_6,growth_future_7,growth_future_8,growth_future_9,growth_future_10,...,growth_future_21,growth_future_22,growth_future_23,growth_future_24,growth_future_25,growth_future_26,growth_future_27,growth_future_28,growth_future_29,growth_future_30
count,130.000000,130.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,128.000000,128.000000,...,118.000000,118.000000,118.000000,115.000000,113.000000,111.000000,111.000000,111.000000,110.000000,108.000000
mean,0.936904,0.926038,0.921692,0.916608,0.900058,0.894096,0.879467,0.878614,0.885019,0.879312,...,0.906514,0.900841,0.900500,0.890874,0.876924,0.886971,0.865510,0.872173,0.865879,0.865879
std,0.183107,0.221683,0.274056,0.291695,0.326447,0.343763,0.332339,0.361255,0.401563,0.417606,...,0.614777,0.580315,0.594037,0.590117,0.597118,0.659818,0.614379,0.609930,0.580562,0.545256
min,0.153569,0.108733,0.086641,0.094257,0.081118,0.087677,0.085261,0.090123,0.095616,0.096997,...,0.048326,0.048326,0.049361,0.048326,0.048326,0.045219,0.044529,0.046945,0.042803,0.040387
25%,0.875264,0.814060,0.829016,0.818482,0.705376,0.716129,0.705714,0.686651,0.634048,0.623677,...,0.525153,0.530734,0.518917,0.500653,0.500000,0.488057,0.468011,0.474236,0.485379,0.491851
50%,0.982438,0.984850,0.977739,0.961375,0.960000,0.939823,0.947344,0.924938,0.944536,0.924911,...,0.918656,0.934281,0.910967,0.882742,0.868895,0.864800,0.854447,0.837310,0.846838,0.868795
75%,1.025544,1.025667,1.020690,1.028182,1.020985,1.029032,1.022305,1.030422,1.030866,1.028701,...,1.039435,1.046383,1.044891,1.066591,1.044959,1.047534,1.051465,1.058080,1.057249,1.054627
max,1.362069,1.464015,2.380000,2.083710,2.262443,2.529870,2.173913,2.359740,2.751948,3.176087,...,4.500000,3.871041,3.846154,3.803394,3.427273,4.817886,4.434146,4.570732,4.247155,3.629268


In [19]:
quartiles_75 = stacked_ipos_df.drop(["IPO Price", "Current", "Return"], axis=1).describe().loc["75%"]
quartiles_75.sort_values(ascending=True)

growth_future_3     1.020690
growth_future_5     1.020985
growth_future_7     1.022305
growth_future_1     1.025544
growth_future_2     1.025667
growth_future_15    1.027022
growth_future_4     1.028182
growth_future_10    1.028701
growth_future_6     1.029032
growth_future_12    1.029173
growth_future_14    1.030035
growth_future_8     1.030422
growth_future_18    1.030597
growth_future_9     1.030866
growth_future_11    1.032077
growth_future_17    1.032338
growth_future_19    1.033840
growth_future_13    1.034919
growth_future_21    1.039435
growth_future_23    1.044891
growth_future_25    1.044959
growth_future_22    1.046383
growth_future_20    1.047235
growth_future_26    1.047534
growth_future_27    1.051465
growth_future_16    1.052174
growth_future_30    1.054627
growth_future_29    1.057249
growth_future_28    1.058080
growth_future_24    1.066591
Name: 75%, dtype: float64

**Growth_Future_24 is the highest**. BUT this is only for **129 stocks**. I'm not sure how to get information for the remaining?

**Q3: Is Growth Concentrated in the Largest Stocks? - Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**
---

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`

Now let's add the top 12-22 stocks (as of end-April 2024):

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate `growth_7d` for every stock and every day. Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

- Date	ticker_category	growth_7d
- 2014-01-01	LARGE	1.011684
- 2014-01-01	LARGEST	1.011797

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [20]:
!pip install yfinance
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import date


In [21]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']
LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']
NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']
NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']
LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [25]:
# From code snippet 7
def gather_stocks_data(ticker_symbols):
    stocks_df = pd.DataFrame({'A' : []})
    for i,ticker in enumerate(ticker_symbols):
        print(i,ticker)

        # Work with stock prices
        historyPrices = yf.download(tickers = ticker,
                            period = "max",
                            interval = "1d")

        # generate features for historical prices, and what we want to predict
        historyPrices['Ticker'] = ticker
        historyPrices['Year']= historyPrices.index.year
        historyPrices['Month'] = historyPrices.index.month
        historyPrices['Weekday'] = historyPrices.index.weekday
        historyPrices['Date'] = historyPrices.index.date

        # historical returns
        for i in [1,3,7,30,90,365]:
            historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
        historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

        # Technical indicators
        # SimpleMovingAverage 10 days and 20 days
        historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
        historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
        historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
        historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

        # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
        historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

        # what we want to predict
        historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

        # sleep 1 sec between downloads - not to overload the API server
        time.sleep(1)


        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)
    return stocks_df

In [26]:
start_date = (2014, 1, 1)
end_date = date(2023, 12, 31)
largest_stocks_data = gather_stocks_data(LARGEST_STOCKS)
large_stocks_data = gather_stocks_data(LARGE_STOCKS)


0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


32 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


33 TITAN.NS


[*********************100%%**********************]  1 of 1 completed


In [27]:
largest_stocks_data

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_30d,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future
0,0.088542,0.101563,0.088542,0.097222,0.060163,1031788800,MSFT,1986,3,3,...,NaN,NaN,NaN,0.982145,NaN,NaN,0,0.216429,NaN,0
1,0.097222,0.102431,0.097222,0.100694,0.062311,308160000,MSFT,1986,3,4,...,NaN,NaN,NaN,0.922418,NaN,NaN,0,0.083596,NaN,0
2,0.100694,0.103299,0.100694,0.102431,0.063386,133171200,MSFT,1986,3,0,...,NaN,NaN,NaN,0.881354,NaN,NaN,0,0.041097,NaN,0
3,0.102431,0.103299,0.098958,0.099826,0.061774,67766400,MSFT,1986,3,1,...,NaN,NaN,NaN,0.921743,NaN,NaN,0,0.070272,NaN,0
4,0.099826,0.100694,0.097222,0.098090,0.060700,47894400,MSFT,1986,3,2,...,NaN,NaN,NaN,0.964604,NaN,NaN,0,0.057199,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221137,3639.000000,3648.949951,3584.050049,3594.300049,3594.300049,1571996,LT.NS,2024,4,1,...,1.015755,1.046955,1.793402,NaN,3595.655005,3677.585010,0,0.018056,1619.620746,0
221138,3590.050049,3634.149902,3576.050049,3599.500000,3599.500000,3748847,LT.NS,2024,5,3,...,0.997589,1.031967,1.790292,NaN,3601.260010,3665.660010,0,0.016141,1621.903216,0
221139,3610.000000,3622.000000,3488.449951,3499.800049,3499.800049,4079696,LT.NS,2024,5,4,...,0.989441,1.002348,1.738504,NaN,3596.145020,3650.280017,0,0.038159,1648.409347,0
221140,3522.800049,3527.000000,3441.100098,3463.300049,3463.300049,2614667,LT.NS,2024,5,0,...,0.972509,0.989811,1.732523,NaN,3590.640015,3634.322522,0,0.024803,1717.911592,0


In [35]:
# Yes, Yuck, I know!
largest_stocks_data[largest_stocks_data.Year>=2014][largest_stocks_data.Month>=1][largest_stocks_data.Weekday>=1][largest_stocks_data.Year<=2023][largest_stocks_data.Month<=12][largest_stocks_data.Weekday<=31].growth_7d.mean()

<ipython-input-35-751e8cb957c0>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  largest_stocks_data[largest_stocks_data.Year>=2014][largest_stocks_data.Month>=1][largest_stocks_data.Weekday>=1][largest_stocks_data.Year<=2023][largest_stocks_data.Month<=12][largest_stocks_data.Weekday<=31].growth_7d.mean()
<ipython-input-35-751e8cb957c0>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  largest_stocks_data[largest_stocks_data.Year>=2014][largest_stocks_data.Month>=1][largest_stocks_data.Weekday>=1][largest_stocks_data.Year<=2023][largest_stocks_data.Month<=12][largest_stocks_data.Weekday<=31].growth_7d.mean()
<ipython-input-35-751e8cb957c0>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  largest_stocks_data[largest_stocks_data.Year>=2014][largest_stocks_data.Month>=1][largest_stocks_data.Weekday>=1][largest_stocks_data.Year<=2023][largest_stocks_data.Month<=12][largest_stocks_data.Weekday<=3

1.0060963859518945

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

In [ ]:
# This requires data wrangling skills currently beyond me, I will guess the homework answers!

**Q4: Trying Another Technical Indicators strategy - What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**
---

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS \$ (closest integer value) over many operations in 10 years? One operation calculations: if you invested \$1000 and received \$1010 in 5 days - you add $10 to gross profit, if you received \$980 - add -\$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

- Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
- Are you still profitable on those trades?